In [16]:
import gc
import json
import os, h5py
import math
import multiprocessing
import numpy as np
import pandas as pd
import torch
import importlib
import logging
from itertools import combinations
from pathlib import Path
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.utils import resample
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

# Custom modules for data handling, balancing, training, evaluation, and model architectures
import modeleval
import avg_ensemble

importlib.reload(modeleval)
importlib.reload(avg_ensemble)

# Import specific functions from custom modules to keep code clean and readable
from modeleval import (
    dh_test_model, nam_dagostino_chi2, get_baseline_hazard_at_timepoints, combined_test_model, evaluate_cif_predictions
)
from avg_ensemble import (
    load_bootstrap_predictions, generate_combinations, get_cif_for_combination, ensemble_cif_arrays, save_metrics_to_individual_json,
    get_processed_combinations, process_combination, process_combination_worker
)

import psutil
torch.cuda.empty_cache()
gc.collect()

0

In [2]:
# Define Constants and Load Datasets
RANDOM_SEED = 12345
N_SPLIT = 2
FEATURE_COLS = ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
DURATION_COL = 'date_from_sub_60'
EVENT_COL = 'endpoint'
CLUSTER_COL = 'key'
TIME_GRID = np.array([i * 365 for i in range(6)])

# Define Feature Groups
CAT_FEATURES = ['gender', 'dm', 'ht', 'sprint']
LOG_FEATURES = ['a1c', 'po4', 'UACR_mg_g', 'Cr']
STANDARD_FEATURES = ['age', 'alb', 'ca', 'hb', 'hco3']
PASSTHROUGH_FEATURES = ['key', 'date_from_sub_60', 'endpoint']

# List of CIF labels
cif_array_labels = [
    "deepsurv_ann_clustering",
    "deepsurv_ann_enn",
    "deepsurv_ann_tomek",
    "deepsurv_lstm_clustering",
    "deepsurv_lstm_NearMiss",
    "deephit_ann_clustering",
    "deephit_ann_NearMiss",
    "deephit_lstm_clustering",
    "deephit_lstm_NearMiss",
]

In [11]:

def parallel_process_combinations(bootstrap_data, all_combinations, output_dir, n_workers=4):
    """
    Parallel processing of combinations across bootstrap iterations.

    Args:
        bootstrap_data (dict): Loaded bootstrap data.
        all_combinations (list): List of all combinations.
        output_dir (str): Directory to save metrics JSON files.
        n_workers (int): Number of parallel workers.

    Returns:
        None
    """
    processed_combinations = get_processed_combinations(output_dir)
    tasks = []

    for i, key in enumerate(sorted(bootstrap_data["predictions"].keys())):
        itr_predictions = bootstrap_data["predictions"][key]
        itr_durations = bootstrap_data["durations"][i]
        itr_events = bootstrap_data["events"][i]

        for combo in all_combinations:
            if (i, combo) in processed_combinations:
                print(f"Skipping already processed Bootstrap {i}, Combination {combo}")
                continue
            tasks.append((i, combo, itr_predictions, itr_durations, itr_events, output_dir))

    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        results = list(executor.map(process_combination_worker, tasks))

    for result in results:
        print(result)

In [17]:
def parallel_process_combinations(bootstrap_data, all_combinations, output_dir, n_workers=4, batch_size=100):
    """
    Process combinations in parallel and save metrics in batches.

    Args:
        bootstrap_data (dict): Data for all bootstrap iterations.
        all_combinations (list): List of all combinations of models.
        output_dir (str): Directory to save metrics.
        n_workers (int): Number of parallel workers.
        batch_size (int): Number of tasks to process per batch.
    """
    processed_combinations = get_processed_combinations(output_dir)

    predictions = bootstrap_data["predictions"]
    durations = bootstrap_data["durations"]
    events = bootstrap_data["events"]

    for i, itr_predictions in enumerate(predictions.values()):
        itr_durations = durations[i]
        itr_events = events[i]

        # Create tasks
        tasks = [
            (i, combo, itr_predictions, itr_durations, itr_events, output_dir)
            for combo in all_combinations
            if (i, combo) not in processed_combinations
        ]

        # Process in batches
        for batch_start in range(0, len(tasks), batch_size):
            batch_tasks = tasks[batch_start:batch_start + batch_size]

            # Use ProcessPoolExecutor for parallel processing of the current batch
            with ProcessPoolExecutor(max_workers=n_workers) as executor:
                results = list(executor.map(process_combination_worker, batch_tasks))
                for result in results:
                    print(result)

In [4]:
# Directory containing the bootstrap HDF5 files
directory_path = '/mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/'
output_dir = "/mnt/d/PYDataScience/g3_regress/data/results/avg_ensemble"

# Load all bootstrap predictions
bootstrap_data = load_bootstrap_predictions(directory_path)
all_combinations = generate_combinations(cif_array_labels)

Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_1.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_10.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_100.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_101.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_102.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_103.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_104.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstrap_iteration_105.h5...
Loading /mnt/d/PYDataScience/g3_regress/data/results/bootstrap_predictions/20241130/bootstr

In [ ]:
# Run parallel processing
parallel_process_combinations(bootstrap_data, all_combinations, output_dir, n_workers=10, batch_size=100)

Error calculating Nam-D'Agostino Chi² at time 0 for event 1: Values must be numeric: no strings, datetimes, objects, etc.


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Error calculating Nam-D'Agostino Chi² at time 0 for event 2: Values must be numeric: no strings, datetimes, objects, etc.


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Metrics for Bootstrap 68, Combination ('deepsurv_lstm_clustering', 'deephit_lstm_NearMiss') saved to /mnt/d/PYDataScience/g3_regress/data/results/avg_ensemble/bootstrap_68_combo_deepsurv_lstm_clustering_deephit_lstm_NearMiss.json.
Metrics for Bootstrap 68, Combination ('deepsurv_lstm_NearMiss', 'deephit_lstm_NearMiss') saved to /mnt/d/PYDataScience/g3_regress/data/results/avg_ensemble/bootstrap_68_combo_deepsurv_lstm_NearMiss_deephit_lstm_NearMiss.json.


...

/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Metrics for Bootstrap 68, Combination ('deepsurv_ann_tomek', 'deephit_ann_clustering', 'deephit_ann_NearMiss') saved to /mnt/d/PYDataScience/g3_regress/data/results/avg_ensemble/bootstrap_68_combo_deepsurv_ann_tomek_deephit_ann_clustering_deephit_ann_NearMiss.json.


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event

Metrics for Bootstrap 68, Combination ('deepsurv_ann_tomek', 'deephit_ann_clustering', 'deephit_lstm_NearMiss') saved to /mnt/d/PYDataScience/g3_regress/data/results/avg_ensemble/bootstrap_68_combo_deepsurv_ann_tomek_deephit_ann_clustering_deephit_lstm_NearMiss.json.


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/aalen_johansen_fitter.py:112: Warning: Tied event times were detected. The Aalen-Johansen estimator cannot handle tied event times.
                To resolve ties, data is randomly jittered.
  warnings.warn(
